# Part 2B: Low level API

Goal:
-----

explore h5py functionality to directly access the low level HDF5 API in Python.

h5py exposes the HDF5 low level API, which can be mixed with the high level interface.

[The HDF5 low level API](https://support.hdfgroup.org/HDF5/doc/RM/RM_H5Front.html) is subdivided into sections:
 * H5A : Attributes
 * H5D : Datasets
 * H5F : File interface

(...)

h5py wrap these functions in cython modules:
 * h5py.h5a
 * h5py.h5d
 * h5py.hdf
 
(...)

Most of the wrappers function live in a proxy class. For example the `H5D` functions are contained in the `DatasetID` class.

h5py low level API reference: http://api.h5py.org/index.html

In [3]:
import h5py
import numpy as np

In [90]:
f = h5py.File('dataset.h5', 'w')
f['/data'] = np.arange(100, dtype=np.int8)
f['/data'][:]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], dtype=int8)

### H5Dget_storage_size() : get size of dataset on disk
```
H5Dget_storage_size
hsize_t H5Dget_storage_size( hid_t dataset_id )
```
HDF5 API reference: https://support.hdfgroup.org/HDF5/doc/RM/RM_H5D.html#Dataset-GetStorageSize

h5py API docs: http://api.h5py.org/h5d.html

In [92]:
dset = f['data']
dset

<HDF5 dataset "data": shape (100,), type "|i1">

In [93]:
dset.id

The `DatasetID` class is a proxy object that wrap the H5D API in the Cython module `h5d.pyx`:


In [94]:
dset.id.get_storage_size()

100L

In [95]:
f.close()

# H5Lcreate_external

Create an external link in a HDF5 file.

link `link.h5:/links/data` to `dataset.h5:/data`

In [75]:
f2 = h5py.File('link.h5', 'w')

In [76]:
g = f2.create_group('links')

In [79]:
group_id = g.id
group_id

Or using the low level API:

```
H5GOpen
hid_t H5Gopen( hid_t loc_id, const char *name )
```

https://support.hdfgroup.org/HDF5/doc/RM/RM_H5G.html#Group-Open

http://api.h5py.org/h5g.html

In [80]:
h5py.h5g.open(f2.id, b'links')

note the bytestrings in the call to the low level API function.

Low level API:

```
H5Lcreate_external

herr_t H5Lcreate_external( const char *target_file_name, const char *target_obj_name, hid_t link_loc_id, const char *link_name, hid_t lcpl_id, hid_t lapl_id )
```

https://support.hdfgroup.org/HDF5/doc/RM/RM_H5L.html#Link-CreateExternal

http://api.h5py.org/h5l.html#module-h5py.h5l



In [81]:
group_id.links.create_external(b'data', b'dataset.h5', b'/data')

In [82]:
g, list(g)

(<HDF5 group "/links" (1 members)>, [u'data'])

In [83]:
f2['/links/data']

<HDF5 dataset "data": shape (100,), type "|i1">

In [84]:
f2['/links/data'][:]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], dtype=int8)

In [85]:
!rm dataset.h5

In [86]:
f2['/links/data']

KeyError: "Unable to open object (Unable to open external file, external link file name = 'dataset.h5', temp_file_name = 'dataset.h5')"

In [87]:
f2.close()

In [88]:
!h5dump link.h5

HDF5 "link.h5" {
GROUP "/" {
   GROUP "links" {
      EXTERNAL_LINK "data" {
         TARGETFILE "dataset.h5"
         TARGETPATH "/data"
      }
   }
}
}


In [96]:
!h5dump dataset.h5

HDF5 "dataset.h5" {
GROUP "/" {
   DATASET "data" {
      DATATYPE  H5T_STD_I8LE
      DATASPACE  SIMPLE { ( 100 ) / ( 100 ) }
      DATA {
      (0): 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
      (19): 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
      (35): 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
      (51): 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
      (67): 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82,
      (83): 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98,
      (99): 99
      }
   }
}
}
